In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
import dask
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
import glob
import random
import math
import json
import os
import gc

## Configuration

In [2]:
dask.config.set(num_workers=8, scheduler='processes')
random.seed(0)

# Directory where the derived data is stored
DERIVED_DATA_DIR = '../../../data'

# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0011_09082021'

# Scaler to normalize the data
SCALER = MinMaxScaler() # StandardScaler()

# Total number of experiments
N_EXPERIMENTS = 63
# Number of folds for cross-validation
CV = 6
# Experiments for training set (int)
TRAIN_SIZE = 54
# Experiments for test set (int)
TEST_SIZE = 9

assert(TRAIN_SIZE + TEST_SIZE == N_EXPERIMENTS)
assert(TRAIN_SIZE % CV == 0)

In [3]:
experiments_dirs_path = glob.glob(DERIVED_DATA_DIR + '/*/*')
assert(len(experiments_dirs_path) == N_EXPERIMENTS)

## Experiment selection (optional)

In [4]:
experiments_dirs_path_filter = []

exp_params = {}
for exp_path in experiments_dirs_path:
    with open(exp_path + '/parameters.json') as f:
        exp_params[exp_path] = json.load(f)
        
    if int(exp_params[exp_path]['SkinConfig']) == 0 or exp_params[exp_path]['SkinConfig'] == 'NaN':
        experiments_dirs_path_filter.append(exp_path)

In [5]:
print('Final number of experiments:', len(experiments_dirs_path_filter))

Final number of experiments: 40


In [6]:
# Redefine the split of the data for the filtered experiments

# Total number of experiments
N_EXPERIMENTS = 40
# Number of folds for cross-validation
CV = 4
# Experiments for training set (int)
TRAIN_SIZE = 32
# Experiments for test set (int)
TEST_SIZE = 8

assert(TRAIN_SIZE + TEST_SIZE == N_EXPERIMENTS)
assert(TRAIN_SIZE % CV == 0)

In [7]:
experiments_dirs_path = experiments_dirs_path_filter

## Features and target selection

In [8]:
JOINT = 'Ankle'
FORCE_CELLS_PER_JOINT = {
    'Hip': [5, 6],
    'Knee': [3, 4, 7, 8],
    'Ankle': [1, 2]
}

H3_LEG = 'L' # L|R

# features = [H3_LEG + a + m for a in ['Hip', 'Knee', 'Ankle'] for m in ['Pos', 'Vel', 'Acc', 'Torque']] + ['LegKnee{}Filtered'.format(m) for m in ['Position', 'Velocity', 'Torque']]
# targets = ['F' + str(i + 1) + ax for i in range(N_CELLS) for ax in ['x', 'y', 'z']]
features = ['L{}Pos'.format(JOINT)] + ['F' + str(i) + 'z' for i in FORCE_CELLS_PER_JOINT[JOINT]]
targets = ['F' + str(i) + ax for i in FORCE_CELLS_PER_JOINT[JOINT] for ax in ['x', 'y']]

print('Number of features: {}'.format(len(features)))
print('Selected features: {}'.format(features))
print('\n')
print('Number of targets: {}'.format(len(targets)))
print('Selected targets: {}'.format(targets))

Number of features: 3
Selected features: ['LAnklePos', 'F1z', 'F2z']


Number of targets: 4
Selected targets: ['F1x', 'F1y', 'F2x', 'F2y']


In [9]:
# Index to crop the data and use only this section of each experiment (start idx, end idx)
# The indexes can be defined manually defining crop_by_index=(start idx, end idx) or seleted at random setting crop_by_index=True
window_size = 200
crop_by_index = False #(1500, 1700) # True

# Sample the experiment data to use only this sample datapoints
random_sample = True
random_sample_pct = 0.05

targets_dict = {}
features_dict = {}
for i, exp_path in enumerate(experiments_dirs_path):
    print('{} - Experiment {} from {}'.format(i, exp_path.split('/')[-1], exp_path.split('/')[-2]))
    
    # Load targets
    targets_df = pd.read_csv(exp_path + '/force_cells_processed.csv')
    
    # Load features
    exo_df = pd.read_csv(exp_path + '/H3_processed.csv')
    # leg_df = pd.read_csv(exp_path + '/leg_processed.csv')
    # features_df = pd.concat([exo_df, leg_df], axis=1)
    features_df = exo_df
    
    idx_aux = targets_df.duplicated(keep='first')
    targets_df = targets_df.loc[~idx_aux]
    features_df = features_df.loc[~idx_aux]
    print('Droping {} duplicated data points'.format(len(idx_aux[idx_aux == False])))
    
    # Drop first row to remove noise in the start of the data recording
    targets_df = targets_df.iloc[1:]
    features_df = features_df.iloc[1:]
    # Drop null values
    idx = features_df.notna().all(axis=1)
    features_df = features_df.loc[idx]
    targets_df = targets_df.loc[idx]
    print('Droping {} data points by null features'.format(len(idx[idx == False])))

    assert(len(features_df) == len(targets_df))
    data_df = pd.concat([features_df, targets_df], axis=1)
    
    # Crop the data by the indicated indexes
    if crop_by_index:
        if crop_by_index == True:
            start_idx = random.randint(100, len(data_df) - window_size - 100)
            crop_by_index = (start_idx, start_idx + window_size)
            
        data_df = data_df.iloc[crop_by_index[0]:crop_by_index[1]]
        
    if random_sample:
        data_df = data_df.sample(frac=random_sample_pct, random_state=0)
        
    # Store the final array
    targets_dict[i] = data_df[targets].values
    features_dict[i] = data_df[features].values
    
    print('Experiment {} -> X: {}, Y: {} \n'.format(i, features_dict[i].shape, targets_dict[i].shape))

0 - Experiment 1 from 10032021
Droping 2917 duplicated data points
Droping 0 data points by null features
Experiment 0 -> X: (146, 3), Y: (146, 4) 

1 - Experiment 1 from 16022021
Droping 8709 duplicated data points
Droping 0 data points by null features
Experiment 1 -> X: (435, 3), Y: (435, 4) 

2 - Experiment 2 from 16022021
Droping 8696 duplicated data points
Droping 0 data points by null features
Experiment 2 -> X: (435, 3), Y: (435, 4) 

3 - Experiment 3 from 16022021
Droping 8708 duplicated data points
Droping 0 data points by null features
Experiment 3 -> X: (435, 3), Y: (435, 4) 

4 - Experiment 2 from 17022021
Droping 8711 duplicated data points
Droping 0 data points by null features
Experiment 4 -> X: (436, 3), Y: (436, 4) 

5 - Experiment 1 from 19022021
Droping 8697 duplicated data points
Droping 0 data points by null features
Experiment 5 -> X: (435, 3), Y: (435, 4) 

6 - Experiment 10 from 19022021
Droping 8725 duplicated data points
Droping 0 data points by null features

## Normalization and split for cross-validation

In [10]:
experiments = list(range(N_EXPERIMENTS))
random.shuffle(experiments)

train_experiments = experiments[:TRAIN_SIZE]
test_experiments = experiments[TRAIN_SIZE:]

print('Train experiments ids ({}): {}'.format(len(train_experiments), train_experiments))
print('Test experiments ids ({}): {}'.format(len(test_experiments), test_experiments))

assert(len(train_experiments) + len(test_experiments) == N_EXPERIMENTS)
# Check that no test experiment is in train
assert(not any([i in test_experiments for i in train_experiments]))

Train experiments ids (32): [0, 27, 12, 21, 29, 15, 38, 34, 1, 20, 7, 5, 10, 13, 14, 37, 22, 39, 23, 17, 8, 35, 3, 33, 9, 4, 36, 6, 28, 18, 11, 30]
Test experiments ids (8): [19, 25, 31, 32, 16, 2, 26, 24]


In [11]:
X_train = np.concatenate([features_dict[i] for i in train_experiments], axis=0)
Y_train = np.concatenate([targets_dict[i] for i in train_experiments], axis=0)
X_test = np.concatenate([features_dict[i] for i in test_experiments], axis=0)
Y_test = np.concatenate([targets_dict[i] for i in test_experiments], axis=0)

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))
print('Total data points: {}'.format(X_train.shape[0] + X_test.shape[0]))

Train -> X: (13633, 3), Y: (13633, 4)
Test -> X: (3466, 3), Y: (3466, 4)
Total data points: 17099


In [12]:
s = SCALER.fit(X_train)

X_train_norm = s.transform(X_train)
X_test_norm = s.transform(X_test)

print('Train -> \n min: {}, \n max: {}, \n mean: {}, \n std: {}\n'.format(np.min(X_train_norm, axis=0), np.max(X_train_norm, axis=0), np.mean(X_train_norm, axis=0), np.std(X_train_norm, axis=0)))
print('Test -> \n min: {}, \n max: {}, \n mean: {}, \n std: {}\n'.format(np.min(X_test_norm, axis=0), np.max(X_test_norm, axis=0), np.mean(X_test_norm, axis=0), np.std(X_test_norm, axis=0)))

Train -> 
 min: [0. 0. 0.], 
 max: [1. 1. 1.], 
 mean: [0.60676458 0.85645142 0.20431342], 
 std: [0.23579962 0.14214477 0.12623251]

Test -> 
 min: [ 0.00507874  0.00639473 -0.05341807], 
 max: [1.00020325 0.97568065 0.75987633], 
 mean: [0.60369541 0.83270744 0.23239737], 
 std: [0.23922545 0.12833765 0.12092265]



In [13]:
save_dir = os.path.join(RESULTS_PATH, DATA_ID, 'data')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

np.save(save_dir + '/' + JOINT + '_X_train_' + DATA_ID + '.npy', X_train_norm)    
np.save(save_dir + '/' + JOINT + '_X_test_' + DATA_ID + '.npy', X_test_norm)    
np.save(save_dir + '/' + JOINT + '_Y_train_' + DATA_ID + '.npy', Y_train)    
np.save(save_dir + '/' + JOINT + '_Y_test_' + DATA_ID + '.npy', Y_test)    

In [14]:
# Split the experiments of the training sets in different folds
exp_per_fold = len(train_experiments) // CV
cv_folds = [train_experiments[x:x+exp_per_fold] for x in range(0, len(train_experiments), exp_per_fold)]
print('CV folds ({}): {}\n'.format(len(cv_folds), cv_folds))

for fold_id in range(CV):
    print('Fold {}'.format(fold_id + 1))
    
    cv_folds_cp = cv_folds.copy()
    valid_experiments_fold = cv_folds_cp.pop(fold_id)
    train_experiments_fold = [item for sublist in cv_folds_cp for item in sublist]

    print('Train experiments ids ({}): {}'.format(len(train_experiments_fold), train_experiments_fold))
    print('Validation experiments ids ({}): {}'.format(len(valid_experiments_fold), valid_experiments_fold))

    assert(len(train_experiments_fold) + len(valid_experiments_fold) == len(train_experiments))
    # Check that no validation experiments are in train
    assert(not any([i in valid_experiments_fold for i in train_experiments_fold]))
    # Check that no test experiments are in train or validation folds
    assert(not any([i in test_experiments for i in train_experiments_fold]))
    assert(not any([i in test_experiments for i in valid_experiments_fold]))
    
    X_train = np.concatenate([features_dict[i] for i in train_experiments_fold], axis=0)
    Y_train = np.concatenate([targets_dict[i] for i in train_experiments_fold], axis=0)
    X_valid = np.concatenate([features_dict[i] for i in valid_experiments_fold], axis=0)
    Y_valid = np.concatenate([targets_dict[i] for i in valid_experiments_fold], axis=0)

    print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
    print('Validation -> X: {}, Y: {}'.format(X_valid.shape, Y_valid.shape))
    
    # Normalize the data
    s = SCALER.fit(X_train)

    X_train_norm = s.transform(X_train)
    X_valid_norm =  s.transform(X_valid)

    print('Train -> \n min: {}, \n max: {}, \n mean: {}, \n std: {}\n'.format(np.min(X_train_norm, axis=0), np.max(X_train_norm, axis=0), np.mean(X_train_norm, axis=0), np.std(X_train_norm, axis=0)))
    print('Valid -> \n min: {}, \n max: {}, \n mean: {}, \n std: {}'.format(np.min(X_valid_norm, axis=0), np.max(X_valid_norm, axis=0), np.mean(X_valid_norm, axis=0), np.std(X_valid_norm, axis=0)))
    
    save_dir = os.path.join(RESULTS_PATH, DATA_ID, 'data')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    np.save(save_dir + '/{}_X_train_cv{}_{}.npy'.format(JOINT, fold_id + 1, DATA_ID), X_train_norm)    
    np.save(save_dir + '/{}_X_valid_cv{}_{}.npy'.format(JOINT, fold_id + 1, DATA_ID), X_valid_norm)    
    np.save(save_dir + '/{}_Y_train_cv{}_{}.npy'.format(JOINT, fold_id + 1, DATA_ID), Y_train)    
    np.save(save_dir + '/{}_Y_valid_cv{}_{}.npy'.format(JOINT, fold_id + 1, DATA_ID), Y_valid)      
    
    print('\n')

CV folds (4): [[0, 27, 12, 21, 29, 15, 38, 34], [1, 20, 7, 5, 10, 13, 14, 37], [22, 39, 23, 17, 8, 35, 3, 33], [9, 4, 36, 6, 28, 18, 11, 30]]

Fold 1
Train experiments ids (24): [1, 20, 7, 5, 10, 13, 14, 37, 22, 39, 23, 17, 8, 35, 3, 33, 9, 4, 36, 6, 28, 18, 11, 30]
Validation experiments ids (8): [0, 27, 12, 21, 29, 15, 38, 34]
Train -> X: (10431, 3), Y: (10431, 4)
Validation -> X: (3202, 3), Y: (3202, 4)
Train -> 
 min: [0. 0. 0.], 
 max: [1. 1. 1.], 
 mean: [0.6029747  0.83271775 0.1924386 ], 
 std: [0.23844401 0.17212837 0.12258567]

Valid -> 
 min: [-0.0105173  -0.21309356  0.03870377], 
 max: [0.99924993 0.99018571 0.81569907], 
 mean: [0.601502   0.80352896 0.24299746], 
 std: [0.23773973 0.17154007 0.13013385]


Fold 2
Train experiments ids (24): [0, 27, 12, 21, 29, 15, 38, 34, 22, 39, 23, 17, 8, 35, 3, 33, 9, 4, 36, 6, 28, 18, 11, 30]
Validation experiments ids (8): [1, 20, 7, 5, 10, 13, 14, 37]
Train -> X: (10152, 3), Y: (10152, 4)
Validation -> X: (3481, 3), Y: (3481, 4)
Tra